In [13]:
# standar imports
import json
import pandas as pd
import re
import tqdm
import pprint as pp

In [14]:
# load the dataset from causalbenchmark/data/cladder-v1-questions.json
with open('../../data/cladder-v1-questions.json') as f:
    data = json.load(f)

In [24]:
# read the dataset as a pandas dataframe
df = pd.DataFrame(data)
df.head()   

,question_id,desc_id,given_info,question,answer,meta,reasoning
0,19,alarm-mediation-nie-model1-spec1-q1,For husbands that don't set the alarm and wive...,Does husband negatively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...
1,23,alarm-mediation-marginal-model1-spec1-q1,The overall probability of alarm set by husban...,Is ringing alarm less likely than silent alarm...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...
2,35,alarm-mediation-nde-model2-spec2-q1,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...
3,51,alarm-mediation-marginal-model3-spec3-q1,The overall probability of alarm set by husban...,Is ringing alarm less likely than silent alarm...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...
4,54,alarm-mediation-backadj-model3-spec3-q0,Method 1: We look at how husband correlates wi...,"To understand how husband affects alarm clock,...",no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",None


In [37]:
df_reasoning = df['reasoning']
d = df_reasoning.head()[0]
reasoning = ' '.join([str(k) + ": " +  str(v) for k, v in d.items()])
# replace the new line characters with spaces
reasoning = re.sub('\n', ', ', reasoning)
reasoning

'step0: Let X = husband; V2 = wife; Y = alarm clock. step1: X->V2,X->Y,V2->Y step2: E[Y_{X=0, V2=1} - Y_{X=0, V2=0}] step3: \\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X = 1) − P(V2 = v | X = 0)] step4: P(Y=1 | X=0, V2=0) = 0.06, P(Y=1 | X=0, V2=1) = 0.51, P(Y=1 | X=1, V2=0) = 0.47, P(Y=1 | X=1, V2=1) = 0.92, P(V2=1 | X=0) = 0.45, P(V2=1 | X=1) = 0.63 step5: 0.63 * (0.51 - 0.06)+ 0.45 * (0.92 - 0.47)= 0.08 end: 0.08 > 0'

In [38]:
# write each row of the dataframe to a file 
with open('../../data/cladder-v1-reasoning.txt', 'w') as f:
    for index, row in tqdm.tqdm(df.iterrows()):
        
        d: dict = row['reasoning']
        if d is not None:
            # concat the reasoning dict entries into a single string
            reasoning = ' '.join([str(k) + ": " +  str(v) for k, v in d.items()])
            reasoning = re.sub('\n', ', ', reasoning)
            f.write(reasoning + '\n')

10560it [00:00, 49871.75it/s]


In [39]:
# write each row of the dataframe to a file 
with open('../../data/cladder-v1-question_reasoning.txt', 'w') as f:
    for index, row in tqdm.tqdm(df.iterrows()):
        d: dict = row['reasoning']
        if d is not None:
            # concat the reasoning dict entries into a single string
            reasoning = ' '.join([str(k) + ": " +  str(v) for k, v in d.items()])
            reasoning = re.sub('\n', ', ', reasoning)
            f.write(row['given_info'] + ' ' + row['question'] + ' '+ reasoning + '\n')
        else:
            f.write(row['given_info'] + ' ' + row['question'] + '\n')

10560it [00:00, 39950.54it/s]


In [5]:
# keep the given_info and question columns
df = df[['given_info', 'question']]
df.head()

,given_info,question
0,For husbands that don't set the alarm and wive...,Does husband negatively affect alarm clock thr...
1,The overall probability of alarm set by husban...,Is ringing alarm less likely than silent alarm...
2,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...
3,The overall probability of alarm set by husban...,Is ringing alarm less likely than silent alarm...
4,Method 1: We look at how husband correlates wi...,"To understand how husband affects alarm clock,..."


In [19]:
# write each row of the dataframe to a file 
with open('../../data/cladder-v1-questions.txt', 'w') as f:
    for index, row in tqdm.tqdm(df.iterrows()):
        f.write(row['given_info'] + ' ' + row['question'] + '\n')


10560it [00:00, 54765.81it/s]


In [ ]:
#@title define functions
params = {
    'max_length':1024,
    'repetition_penalty':1.05,
    'num_beams':4
}
def split_text(text: str) -> list:
    # Split the text into sentences using regex
    sentences = re.split(r"(?<=[^A-Z].[.?]) +(?=[A-Z])", text)
    sentence_batches = []
    temp_batch = []
    for sentence in sentences:
        temp_batch.append(sentence)
        """If the length of the temporary batch is between 2 
        and 3 sentences, or if it is the last batch, add 
        it to the list of sentence batches """
        if len(temp_batch) >= 2 and len(temp_batch) <= 3 or sentence == sentences[-1]:
            sentence_batches.append(temp_batch)
            temp_batch = []
    return sentence_batches

In [ ]:
def correct_text(text: str, checker, corrector, separator: str = " ") -> str:
    sentence_batches = split_text(text)
    corrected_text = []
    for batch in tqdm(
        sentence_batches, total=len(sentence_batches), desc="correcting text.."
    ):
        raw_text = " ".join(batch)
        results = checker(raw_text)
        if results[0]["label"] != "LABEL_1" or (
            results[0]["label"] == "LABEL_1" and results[0]["score"] < 0.9
        ):
            # Correct the text using the text-generation pipeline
            corrected_batch = corrector(raw_text, **params)
            corrected_text.append(corrected_batch[0]["generated_text"])
            print("-----------------------------")
        else:
            corrected_text.append(raw_text)
    corrected_text = separator.join(corrected_text)
    return corrected_text

In [ ]:
# Initialize the text-classification pipeline
from transformers import pipeline
checker = pipeline("text-classification", "textattack/roberta-base-CoLA")

# Initialize the text-generation pipeline
from transformers import pipeline
corrector = pipeline(
        "text2text-generation",
        "pszemraj/flan-t5-large-grammar-synthesis",
         device=0
    )

In [ ]:
raw_text = "my helth is not well, I hv to tak 2 day leave."
# pp.pprint(raw_text)
corrected_text = correct_text(raw_text, checker, corrector)
pp.pprint(corrected_text)

# output:
#'my health is not well, I have to take 2 days leave.'